<h1> Getting started with tf.learn </h1>

In [19]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

Read data created in Lab1a.

In [21]:
def read_dataset(filename):
  return pd.read_csv(filename, header=None, names=['hour','dayofweek','pickuplon','pickuplat','dropofflon','dropofflat','passengers','fare_amount'])

df_train = read_dataset('../lab1a/taxi-train.csv')
df_test = read_dataset('../lab1a/taxi-test.csv')
df_train[:5]

,hour,dayofweek,pickuplon,pickuplat,dropofflon,dropofflat,passengers,fare_amount
0,6,2,-73.989192,40.748615,-73.970180,40.756380,1,12.00
1,6,2,-73.974967,40.735102,-73.776493,40.644974,3,49.57
2,17,5,-73.995045,40.725998,-74.004505,40.734823,1,6.10
3,11,0,-73.967158,40.772232,-73.991385,40.748908,1,12.00
4,16,0,-73.977785,40.752055,-73.979140,40.762352,5,6.50


<h3> Linear Regression with tf.learn Estimators framework </h3>

In [28]:
tf.logging.set_verbosity(tf.logging.INFO)
predictors = df_train.iloc[:,0:7].values # np.ndarray
targets = df_train.iloc[:,7].values
features = tf.contrib.learn.infer_real_valued_columns_from_input(predictors)
shutil.rmtree('taxi_model', ignore_errors=True) # start fresh each time
model = tf.contrib.learn.LinearRegressor(feature_columns=features, model_dir='taxi_model')
model.fit(predictors, targets, steps=1000)

INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=7, default_value=None, dtype=tf.float32)
INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Step 1: loss = 215.62
INFO:tensorflow:Step 101: loss = 88.8777
INFO:tensorflow:Step 201: loss = 88.8666
INFO:tensorflow:Saving checkpoints for 300 into taxi_model/model.ckpt.
INFO:tensorflow:Step 301: loss = 88.8633
INFO:tensorflow:Step 401: loss = 88.8614
INFO:tensorflow:Step 501: loss = 88.8599
INFO:tensorflow:Saving checkpoints for 600 into taxi_model/model.ckpt.
INFO:tensorflow:Step 601: loss = 88.8587
INFO:tensorflow:Step 701: loss = 88.8576
INFO:tensorflow:Step 801: loss = 88.8567
INFO:tensorflow:Saving checkpoints for 900 into taxi_model/model.ckpt.
INFO:tensorflow:Step 901: loss = 88.8559
INFO:tensorflow:Saving checkpoints for 1000 into taxi_model/model.ckpt.
INFO:tensorflow:Loss for final step: 88.8551.


LinearRegressor()

Evaluate on test data (we didn't use validation data)

In [29]:
def compute_rmse(actual, predicted):
  return np.sqrt(np.mean((actual-predicted)**2))

print "Train RMSE = {0}".format(compute_rmse(df_train['fare_amount'], model.predict(df_train.iloc[:,0:7].values)))
print "Test RMSE = {0}".format(compute_rmse(df_test['fare_amount'], model.predict(df_test.iloc[:,0:7].values)))

INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=7, default_value=None, dtype=tf.float32)
INFO:tensorflow:Loading model from checkpoint: taxi_model/model.ckpt-1000-?????-of-00001.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=7, default_value=None, dtype=tf.float32)
INFO:tensorflow:Loading model from checkpoint: taxi_model/model.ckpt-1000-?????-of-00001.


Train RMSE = 9.42629859838
Test RMSE = 8.93285798461


<h3> Deep Neural Network regression </h3>

In [32]:
shutil.rmtree('taxi_model', ignore_errors=True) # start fresh each time
model = tf.contrib.learn.DNNRegressor(feature_columns=features, hidden_units=[128, 8], model_dir='taxi_model')
model.fit(predictors, targets, steps=1000)
print "Train RMSE = {0}".format(compute_rmse(df_train['fare_amount'], model.predict(df_train.iloc[:,0:7].values)))
print "Test RMSE = {0}".format(compute_rmse(df_test['fare_amount'], model.predict(df_test.iloc[:,0:7].values)))

INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=7, default_value=None, dtype=tf.float32)
INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Step 1: loss = 1127.37
INFO:tensorflow:Step 101: loss = 88.9023
INFO:tensorflow:Step 201: loss = 88.8968
INFO:tensorflow:Saving checkpoints for 300 into taxi_model/model.ckpt.
INFO:tensorflow:Step 301: loss = 88.8929
INFO:tensorflow:Step 401: loss = 88.8899
INFO:tensorflow:Step 501: loss = 88.8875
INFO:tensorflow:Saving checkpoints for 600 into taxi_model/model.ckpt.
INFO:tensorflow:Step 601: loss = 88.8854
INFO:tensorflow:Step 701: loss = 88.8836
INFO:tensorflow:Step 801: loss = 88.8821
INFO:tensorflow:Saving checkpoints for 900 into taxi_model/model.ckpt.
INFO:tensorflow:Step 901: loss = 88.8807
INFO:tensorflow:Saving checkpoints for 1000 into taxi_model/model.ckpt.
INFO:tensorflow:Loss for final step: 88.8794.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=7, de

Train RMSE = 9.42758864925
Test RMSE = 8.92909041917


We are not quite beating our benchmark yet ... what's up?  Well, we are using TensorFlow for Machine Learning, but we are not yet using it well.